# Traffic Data Analysis

This is a preliminary analysis of traffic crash data made available at 
https://opendata-nzta.opendata.arcgis.com/datasets/crash-analysis-system-cas-data-1/explore?location=-9.510847%2C0.000000%2C2.92

Note that this dataset is not static, therefore its contents may have changed since the time it was downloaded for the purpose of this analysis.

The goal is to
- Characterize the dataset.
- Discern basic trends concerning crash metrics contained in the dataset.


In [1]:
# Load libraries
import pandas as pd
import numpy as np
import os

In [10]:
# set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# Define functions
def get_dataset_stats(df, exclude=["excluded_col_1","excluded_col_2"], round_digits=3):
    """
    PARAMETERS:
        df - A dataframe 
    
    RETURNS: a dataframe
    """


    include_columns = list(df.columns)
    for element in exclude:
        if element in include_columns:
            include_columns.remove(element)
    
    
    types_df = pd.DataFrame(df[include_columns].dtypes).rename({0:"dtype"}, axis=1)
    obj_list = df[include_columns].select_dtypes(include=['object']).columns.to_list()
    # local function to determine new datatypes for object columns:
    def get_obj_type(df, var):
        type = ""
        if df[var].isnull().all():
            type = 'none'
        else:
            try:
                df[var].astype("datetime64[ns]")
                type = 'datetime64[ns]'
            except:
                try:
                    df[var].astype("int64")
                    type = 'int64'
                except:
                    try:
                        df[var].astype("float64")
                        type = 'float64'  
                    except:
                        try:
                            df[var].astype("str")
                            type = 'str'
                        except:
                            type = 'str'
                            pass
        return type    
    
    for col in obj_list:
        new_type = get_obj_type(df, col)
        print("{} new type is {}".format(col, new_type))
        types_df["dtype"][col] = new_type
    
    
    dg = pd.DataFrame(np.nan, index=include_columns, 
                      columns=["type","excluded","non_null_count","nullcnt","%nulls","min",
                               "pcntl25","median","mean","pcntl75","max",
                               "stddev","str-maxlen","str-meanlen","str-medianlen","str-minlen","ndistinct"])
    
    try:
        for varname in include_columns:
            if str(types_df["dtype"][varname]) in ["datetime64[ns, psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)]"]:
                df[varname] = df[varname].dt.tz_localize(None)
                types_df["dtype"][varname] = "datetime64[ns]"
                
            dg.type.loc[varname] = types_df.loc[varname, "dtype"]
            if dg.type.loc[varname] == 'float64':
                df[varname] = df[varname].astype('float')
            elif dg.type.loc[varname] == 'int64':
                df[varname] = df[varname].astype('int')
            if varname in exclude:
                dg.loc[varname,"excluded"] = 'Yes'
            else:
                dg.loc[varname,"excluded"] = 'No'
            # print("Var: {}  Type: {}".format(varname, dg.type.loc[varname]))
            dg.loc[varname,"ndistinct"] = df[varname].nunique()

            if types_df["dtype"][varname] in ["float64","int64","int32","datetime64[ns]","str","none"]:
                dg.loc[varname, "nullcnt"] = df[varname].isna().sum().astype(int)
                totalcnt = df[varname].count().astype(int) + dg.loc[varname, "nullcnt"]
                dg.loc[varname,"non_null_count"] = totalcnt - dg.loc[varname, "nullcnt"]
                dg.loc[varname,"%nulls"] = (100.0 * dg.loc[varname,"nullcnt"] / totalcnt).round(decimals=2)
            if types_df["dtype"][varname] in ["datetime64[ns]"]:
                dg.loc[varname,"min"] = df[varname].dropna().min()
                dg.loc[varname,"max"] = df[varname].dropna().max()
                dg.loc[varname,"mean"] = df[varname].dropna().mean()
            elif types_df["dtype"][varname] in ["float64","int64","int32"]:
                dg.loc[varname,"min"] = round(df[varname].dropna().min(), round_digits)
                dg.loc[varname,"max"] = round(df[varname].dropna().max(), round_digits)
                dg.loc[varname,"mean"] = round(df[varname].dropna().mean(), round_digits)                
            if types_df["dtype"][varname] in ["float64","int64","int32"]:
                dg.loc[varname,"pcntl25"] = round(df[varname].dropna().quantile(0.25), round_digits)
                dg.loc[varname,"median"] = round(df[varname].dropna().quantile(0.5), round_digits) 
                dg.loc[varname,"pcntl75"] = round(df[varname].dropna().quantile(0.75), round_digits)
                dg.loc[varname,"stddev"] = round(df[varname].dropna().std(), round_digits)
            if types_df["dtype"][varname] in ["str"] and varname not in exclude:
                dg.loc[varname,"str-maxlen"] = df[varname].str.len().max()
                dg.loc[varname,"str-meanlen"] = df[varname].str.len().mean()
                dg.loc[varname,"str-medianlen"] = df[varname].str.len().median()
                dg.loc[varname,"str-minlen"] = df[varname].str.len().min()
                           
    except:
        print("Failed iteration on variable: {}".format(varname))
    
    return dg

In [3]:
# Define working directories
data_dir = "../data"
output_dir = "../outputs"
os.chdir(data_dir)
os.getcwd()

'D:\\Repos\\Traffic_Data_Study\\data'

In [4]:
## Load data
df = pd.read_csv("Crash_Analysis_System_(CAS)_data.csv")

In [5]:
df.head(6)

,X,Y,OBJECTID,advisorySpeed,areaUnitID,bicycle,bridge,bus,carStationWagon,cliffBank,...,train,tree,truck,unknownVehicleType,urban,vanOrUtility,vehicle,waterRiver,weatherA,weatherB
0,1772561.0,5896382.0,1,NaN,525420.0,0.0,NaN,0.0,1.0,NaN,...,NaN,NaN,0.0,0.0,Urban,0.0,NaN,NaN,Fine,Null
1,1836757.0,5859311.0,3,NaN,534300.0,0.0,NaN,0.0,2.0,NaN,...,NaN,NaN,0.0,0.0,Urban,0.0,NaN,NaN,Fine,Frost
2,1762088.0,5912507.0,4,NaN,519500.0,0.0,NaN,0.0,1.0,NaN,...,NaN,NaN,0.0,0.0,Urban,1.0,NaN,NaN,Fine,Null
3,1753522.0,5911939.0,6,NaN,518902.0,0.0,NaN,0.0,2.0,NaN,...,NaN,NaN,0.0,0.0,Urban,0.0,NaN,NaN,Light rain,Null
4,1761364.0,5914259.0,7,NaN,520202.0,0.0,NaN,0.0,2.0,NaN,...,NaN,NaN,0.0,0.0,Urban,0.0,NaN,NaN,Fine,Null
5,1803464.0,5815043.0,9,NaN,530500.0,0.0,NaN,0.0,3.0,NaN,...,NaN,NaN,0.0,0.0,Urban,0.0,NaN,NaN,Fine,Null


In [6]:
# Obtain basic information: field names, datatypes, non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776878 entries, 0 to 776877
Data columns (total 72 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   X                          776878 non-null  float64
 1   Y                          776878 non-null  float64
 2   OBJECTID                   776878 non-null  int64  
 3   advisorySpeed              29289 non-null   float64
 4   areaUnitID                 776762 non-null  float64
 5   bicycle                    776873 non-null  float64
 6   bridge                     310482 non-null  float64
 7   bus                        776873 non-null  float64
 8   carStationWagon            776873 non-null  float64
 9   cliffBank                  310482 non-null  float64
 10  crashDirectionDescription  776878 non-null  object 
 11  crashFinancialYear         776878 non-null  object 
 12  crashLocation1             776878 non-null  object 
 13  crashLocation2             77

In [7]:
# Produce basic statistical report. Determine viable atomic types for those columns of 'object' Dtype.
df_stats = get_dataset_stats(df)


crashDirectionDescription new type is str
crashFinancialYear new type is str
crashLocation1 new type is str
crashLocation2 new type is str
crashSeverity new type is str
crashSHDescription new type is str
directionRoleDescription new type is str
flatHill new type is str
holiday new type is str
light new type is str
region new type is str
roadCharacter new type is str
roadLane new type is str
roadSurface new type is str
streetLight new type is str
tlaName new type is str
trafficControl new type is str
urban new type is str
weatherA new type is str
weatherB new type is str


D:\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [11]:
# Display computed column metrics for dataframe.
df_stats

,type,excluded,non_null_count,nullcnt,%nulls,min,pcntl25,median,mean,pcntl75,max,stddev,str-maxlen,str-meanlen,str-medianlen,str-minlen,ndistinct
X,float64,No,776878.0,0.0,0.00,1150346.0,1704491.50,1757273.000,1720811.722,1792725.00,2465388.404,153587.370,NaN,NaN,NaN,NaN,300896.0
Y,float64,No,776878.0,0.0,0.00,4793921.0,5433734.00,5803817.226,5644636.177,5914008.00,6190095.000,330871.527,NaN,NaN,NaN,NaN,337344.0
OBJECTID,int64,No,776878.0,0.0,0.00,1.0,318535.25,637330.000,637114.897,955793.75,1274097.000,367820.479,NaN,NaN,NaN,NaN,776878.0
advisorySpeed,float64,No,29289.0,747589.0,96.23,10.0,40.00,55.000,54.504,65.00,95.000,18.213,NaN,NaN,NaN,NaN,18.0
areaUnitID,float64,No,776762.0,116.0,0.01,500100.0,519200.00,536630.000,546213.268,573526.00,626801.000,32591.674,NaN,NaN,NaN,NaN,1875.0
bicycle,float64,No,776873.0,5.0,0.00,0.0,0.00,0.000,0.029,0.00,5.000,0.171,NaN,NaN,NaN,NaN,6.0
bridge,float64,No,310482.0,466396.0,60.03,0.0,0.00,0.000,0.014,0.00,4.000,0.118,NaN,NaN,NaN,NaN,5.0
bus,float64,No,776873.0,5.0,0.00,0.0,0.00,0.000,0.016,0.00,3.000,0.127,NaN,NaN,NaN,NaN,4.0
carStationWagon,float64,No,776873.0,5.0,0.00,0.0,1.00,1.000,1.320,2.00,11.000,0.784,NaN,NaN,NaN,NaN,12.0
cliffBank,float64,No,310482.0,466396.0,60.03,0.0,0.00,0.000,0.106,0.00,3.000,0.309,NaN,NaN,NaN,NaN,4.0


In [12]:
## Find all values for low-cardinality str columns.



## (a) Let's check whether the actual value ranges align with those declared in the MoT codebook. 
##     Of special interest will be nulls, NaNs or obvious proxies for these.
## (b) Some of these features may be candidates for one-hot-encoding if ML models are developed from this dataset.

## We have arbitrarily set the "low cardinality" threshold to 60, though in practice most string columns of interest have
## fewer distinct values, as can be seen from the previous cell.

distinct_vals = pd.DataFrame(data=None, columns=['Variable', 'Value','Count'])
cardinality_thrsld = 60
for col in df_stats[df_stats.type == 'str'][df_stats.ndistinct < cardinality_thrsld].index.values.tolist():
    for val in df[col].unique():
        if pd.isna(val):
            count = len(df[df[col].isnull()]) # Count the occurrence of NaN values if applicable within this column
        else:
            count = len(df[df[col] == val]) # Count the occurrence of this value within this column
        # Append to dataframe
        distinct_vals.loc[len(distinct_vals)] = [col, val, count]

<ipython-input-12-fdf10074304e>:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for col in df_stats[df_stats.type == 'str'][df_stats.ndistinct < cardinality_thrsld].index.values.tolist():


In [13]:
distinct_vals

,Variable,Value,Count
0,crashDirectionDescription,Null,292055
1,crashDirectionDescription,West,95492
2,crashDirectionDescription,North,145423
3,crashDirectionDescription,South,149361
4,crashDirectionDescription,East,94547
5,crashFinancialYear,2007/2008,41119
6,crashFinancialYear,2002/2003,38093
7,crashFinancialYear,2009/2010,37209
8,crashFinancialYear,2017/2018,39583
9,crashFinancialYear,2013/2014,29965
